# Online Shoppers Purchasing Intention Clustering

Isaiah Jenkins

## Import the required libraries

## 1. About the data

1. a. Description

Throughout this analysis we will explore online shoppers purchasing habits and online activity to achieve shopper segmentation. Of the 12,330 sessions in the dataset, 84.5% (10,422) were negative class samples that did not end with shopping, and the rest (1908) were positive class samples ending with shopping.

1. b. Attributes dictionary, 17 features, 10 numerical, 8 categorical

Administrative, Administrative Duration, Informational, Informational Duration, Product Related, Product Related Duration 
- The number of different types of pages visited by the visitor in that session and total time spent in each of these page categories
  
Bounce Rates, Exit Rates, Page Value
- Metrics measured by "Google Analytics" for each page in the e-commerce site. The value of "Bounce Rate" feature for a web page refers to the percentage of visitors who enter the site from that page and then leave ("bounce") without triggering any other requests to the analytics server during that session. The value of "Exit Rate" feature for a specific web page is calculated as for all page views to the page, the percentage that were the last in the session. The "Page Value" feature represents the average value for a web page that a user visited before completing an e-commerce transaction.

Special Day
- Indicates the closeness of the site visiting time to a specific special day (e.g. Mother’s Day, Valentine's Day) in which the sessions are more likely to be finalized with transaction. The value of this attribute is determined by considering the dynamics of e-commerce such as the duration between the order date and delivery date.

Month, Operating Systems, Browser, Traffic Type
- General information

Visitor Type 
- returning or new visitor category

Weekend
- Boolean value indicating whether the date of the visit is weekend
  
Revenue
- Whether shopper generated revenue for company

## 2. Objectives

Throughout this analysis we will explore and build different clustering models to segment between shoppers that did and did not buy products. A possible problems that could occur may include effective clustering because of fewer samples ending with shopping versus no shopping because 84.5% were negative class samples that did not end with shopping. However, with sophisticated clustering models we should be able to obtain proper segmentation.

## 3. Data Exploration, Cleaning and Feature Engineering

### Load in data

### Summary of Models

## 5. Insights and Key Findings

## 6. Next Steps